In [21]:
import pandas as pd
from twython import Twython, TwythonRateLimitError, TwythonError
from time import sleep
import itertools

In [2]:
import pandas as pd
from twython import Twython, TwythonRateLimitError, TwythonError
from time import sleep
import itertools


In [20]:

APP_KEY = '2701n8yQhIWIGWeld9Pg1pCzo'
APP_SECRET = 'wi9V8ejZdl7MhW8S1Xv2bzqwry8AamEugU0LMFb1w0gxS4imgL'
OAUTH_TOKEN = '104434883-8GlZB2wm7Vn3XKakFBnPlXl2ktgQWTpqC462gTW4'
OAUTH_SECRET = '8nOct9wjzZ8bCgrWbttRW1ksoRXwpOOXqs8c5uuF7ZmPy'

csv_path = 'tweet_ids_and_sentiments.csv'

In [3]:
def paginate(iterable, page_size):
    while True:
        i1, i2 = itertools.tee(iterable)
        iterable, page = (itertools.islice(i1, page_size, None),
                list(itertools.islice(i2, page_size)))
        if len(page) == 0:
            break
        yield page

In [4]:
def get_twython_client():
    return Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_SECRET)

def get_twitter_posts_from_list(client, post_id_list):
    return client.lookup_status(id=post_id_list)

def get_all_tweets(csv_path):
    tweet_ids_and_sentiments = pd.read_csv(csv_path)
    
    twitter_client = get_twython_client()

    pages = list(paginate(tweet_ids_and_sentiments.tweet_id, 100))

    all_tweets = []

    for page in pages:
        try:
            posts = get_twitter_posts_from_list(twitter_client, page)
        except TwythonRateLimitError as e:
            print 'Whoops, we exceeded the rate limit somehow. Pausing for 15 minutes...'
            sleep(15 * 60 + 1)
            posts = get_twitter_posts_from_list(twitter_client, page)
        except TwythonError as e:
            print e

        all_tweets += posts
        print 'got %d posts from page %d...' % (len(posts), pages.index(page) + 1)
    print 'got all pages - %d posts in total' % (len(all_tweets))
    return all_tweets, tweet_ids_and_sentiments


In [7]:
def get_tweet_dataframe(all_tweets, tweet_ids_and_sentiments):
    # List comprehension to grab just the text and ID
    relevant_tweet_info = [
        {
            'text':t['text'],
            'tweet_id':t['id']
        } for t in all_tweets
    ]
  # Turning that list into a dataframe so it can be merged with the dataframe that has the sentiment of each post
    tweet_texts = pd.DataFrame(relevant_tweet_info)
    df = pd.merge(tweet_ids_and_sentiments, tweet_texts)    
    return df


  

In [9]:
def fetch_the_data():
    all_tweets , tweet_ids_and_sentiments = get_all_tweets(csv_path)
    df = get_tweet_dataframe(all_tweets, tweet_ids_and_sentiments)
    return df

In [18]:
#return search results

from twython import Twython, TwythonError


twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_SECRET)
try:
    search_results = twitter.search(q='WebsDotCom', count=50)
except TwythonError as e:
    print e

for tweet in search_results['statuses']:
    print 'Tweet from @%s Date: %s' % (tweet['user']['screen_name'].encode('utf-8'),
                                       tweet['created_at'])
print tweet['text'].encode('utf-8'), '\n'

Tweet from @ungreeninac1978 Date: Sun Aug 27 12:08:49 +0000 2017
Tweet from @Ashley86507987 Date: Wed Aug 23 17:33:23 +0000 2017
RT @inneke1982: Just updated my @websdotcom site. Check it out! http://sneleurocent.webs.com/ 



In [19]:
from twython import TwythonStreamer


class MyStreamer(TwythonStreamer):
    def on_success(self, data):
        if 'text' in data:
            print data['text'].encode('utf-8')
        # Want to disconnect after the first result?
        # self.disconnect()

    def on_error(self, status_code, data):
        print status_code, data

# Requires Authentication as of Twitter API v1.1
stream = MyStreamer(APP_KEY, APP_SECRET,
                    OAUTH_TOKEN, OAUTH_SECRET)

stream.statuses.filter(track='twitter')
# stream.user()
# Read the authenticated users home timeline
# (what they see on Twitter) in real-time
# stream.site(follow='twitter')

RT @ILOVEYOUOXXY: i'm tryna find that person that every time we get on a track it's a masterpiece type shit. https://t.co/l02IyACkIH
RT @_ihateyall: Police department https://t.co/2fUtkorwt0
Agree❤👌👍 https://t.co/d3NVDsBNxS
RT @Shohoku_E_M_S: 運が良かったなその子もお前も

救命ながら脳外らしいことをした藍沢
金曜日の新垣さんならぬ
#木曜日の藍沢先生 https://t.co/bvAwgBYKvs
RT @abstractnights: WHEN FRANK CAME IN HAHAHA https://t.co/bOzEPQoYOo
RT @swarovskitits: We love an unproblematic woke legend https://t.co/7xXa8QVkkB
RT @ratboypcy: 💫 EXO Repackage Giveaway 💫
- Exol only
- Retweet to enter
- Worldwide
- Closes September 1st, 9pm kst https://t.co/2ugNTjvQCa
RT @aurabogado: #SuspectedLooters http://t.co/V2NSeSiuXU
RT @brianschatz: Devry just settled for 100 million dollars with the FTC for defrauding students. What I'm saying is this seems like… 
I CAN WATCH @Jaclynhill ON MY TV https://t.co/loUfkGPCk6
😍😍😍😍😍😍😍 https://t.co/l0fEQPeNoJ
ja zawsze https://t.co/rqlzaQey9U
RT @AllAboutYul: โอ้ววววว ความรู้ใหม่ คุณลุงพี่ยูลเป็นหมอและเจ้าของโรงพ

RT @Rood513: 💫💫جديدنا حاليا💫💫2

طقم نسائي انيق مكون من 
شنطة  ماركة فرزاتشي 
نظارة ماركة فرزاتشي 
سوارة ماركة فرزاتشي 
السعر ٢٧٠… 
RT @Camila_Cabello: soooooo excited to show u guys my tour doc from the #24kmagicworldtour!!! check it out on @youtube very soon 👀👀👀… 
RT @Elsie_sex: Just need your tongue on my boobs https://t.co/hBJdBCV8CK
RT @PrinceJerundio: Vous aimez bien dire ça, puis quand vous allez dans le sud, vous faites des grands snap et en parler comme si vs et… 
RT @leonino_lima: ajudem nem que seja retweetando 🙏🏻 https://t.co/lbgd5Qdvaq
RT @honestA__: fill up trash bags with water &amp; put them at your door, the pressure of the bags will help keep some of the water ou… 
RT @ASTROARENA: Terima kasih &amp; tahniah kepada kontinjen negara yang telah mewarnai temasya Sukan SEA Edisi ke-29 KL 2017!… 
I believe this theory 100% @Llyric_ https://t.co/b3Ih9CYBDH
RT @Ahhj_Pahhj: @TabriaMajors  https://t.co/MLEbMPIcaL
RT @MaurAntunez: Parece que Cosse es parte del "Plan Atlanta" o es

RT @cinicabeyo: Hj eu to só laurinah https://t.co/WHxjOOSTJc
RT @1942bs: imagine using two tragedies to promote racism https://t.co/OkWpV4rgA7
TV Crew Finds Wet, Exhausted Possum Bracing For Harvey https://t.co/zKCLlnLpxE https://t.co/DatxTtiKbB
RT @PrimeroRiver: ¿Te gustaría ver dirigir a River al Mono Burgos en algún momento?
Dale RT si te copa la idea https://t.co/amfYw5DTeC
استاذ الرياضيات م يغيبب . https://t.co/5HdtrWLS4k
I feel like they never got much credit for this it was a really big deal https://t.co/k9oFjlJoQ6
Kath, któż inny mógłby to być https://t.co/InoOOUGc1a
RT @MLBGIFs: Me today: It’s about time I started eating healthier.

Me tomorrow: https://t.co/5hEOIPh58v
Trailer for upcoming Lynn Ramsey/Joaquin Phoenix film.  Looks good! https://t.co/jrrIrDEvBB
RT @JoueurYouteube: Un an de moins que toi 30 gros matchs reussis de plus que toi aussi mdr https://t.co/zpiY6MGdft
RT @barandbench: Read Central Governments response to Marital Rape before the Delhi High court - https://

@jordyncote https://t.co/QghtANcDiD
RT @kantalow1219: 年に数回しか食べられない柴田さん特製バーガー😇😇😇 https://t.co/up2NJcIZKR
RT @Santi_Vatt: Temazo https://t.co/cmSxYmh4NJ
RT @Thomas1774Paine: Yeah, you've broken so many credible Trump stories. Who is your WH source, the portrait of Teddy Roosevelt? You hea… 
sex scams videos sativa rose first anal https://t.co/Ew2gHJiaTU
I memes myself, gotta love Julius Caesar https://t.co/E5U5wMquZk
RT @EmlakCep: Senin açtığın yolda yürüyoruz, senin izindeyiz! #30AğustosZaferBayram 30 Ağustos Zafer Bayramımız kutlu olsun. https://t.co/4…
RT @sabirnazar1: My cartoon on Eid ul Azha https://t.co/q6TBOlR3aG
RT @DearAuntCrabby: Hey, @seanhannity Did you see THIS!?!? https://t.co/6Noe7j0eiH
CatGM Got it. Just to check, did you definitely watch the video ad that grants 30 minutes ad-free listening in ful… https://t.co/gkZ5ODoS2D
RT @Thekedarni007: https://t.co/oX3Ak7aiuf
RT @ValerieJarrett: Tan suit looks pretty good today, doesn't it? https://t.co/hKapWf92id
RT @FlyYoung_Nigg

KeyboardInterrupt: 

In [22]:
#get user time line

from twython import Twython, TwythonError

# Requires Authentication as of Twitter API v1.1
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_SECRET)
try:
    user_timeline = twitter.get_user_timeline(screen_name='Chris Albon')
except TwythonError as e:
    print e

print user_timeline

[{u'contributors': None, u'truncated': False, u'text': u'@AskCiti Hey! I have a few questions regarding the application for a student credit card at citi. Can you help me out?', u'is_quote_status': False, u'in_reply_to_status_id': None, u'id': 887662465005309952, u'favorite_count': 0, u'source': u'<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', u'retweeted': False, u'coordinates': None, u'entities': {u'symbols': [], u'user_mentions': [{u'id': 82151473, u'indices': [0, 8], u'id_str': u'82151473', u'screen_name': u'AskCiti', u'name': u'Citi'}], u'hashtags': [], u'urls': []}, u'in_reply_to_screen_name': u'AskCiti', u'in_reply_to_user_id': 82151473, u'retweet_count': 0, u'id_str': u'887662465005309952', u'favorited': False, u'user': {u'follow_request_sent': False, u'has_extended_profile': True, u'profile_use_background_image': False, u'default_profile_image': False, u'id': 104434883, u'profile_background_image_url_https': u'https://abs.twimg.com/images/themes/theme15/b

In [31]:
#getting trends

from twython import Twython
twitter = Twython(APP_KEY, APP_SECRET, OAUTH_TOKEN, OAUTH_SECRET)

result = twitter.get_place_trends(id = 23424977)

if result:
    for trend in result[0].get("trends", []):
        
                                                print("{0} \n".format(trend["name"]))

#YoureARealAdultWhen 

#WednesdayWisdom 

#BeautyIn5Words 

#AwfulThemeParkIdeas 

Tomi Lahren 

#TaxReform 

Joe Haden 

No More Heroes 

Mike Leake 

Cam Erving 

Late Registration 

Nick Kyrgios 

Aaron Rodgers 

McCaskill 

Ed Cunningham 

Ender Inciarte 

Louise Hay 

Erik Prince 

Johnny Manziel 

Rick Smith 

Silvio De Sousa 

Golf Story 

The Blackout 

New America 

Ball in the Family 

Shovel Knight 

Gossip Girl 

Tony Romo 

Yasiin Bey 

Columbus Day 

#BadReasonsToQuit 

#PortArthur 

#nextchat 

#Irma 

#NindiesShowcase 

#AdweekChat 

#WithDreamers 

#WellnessWed 

#SpringsteenOnBroadway 

#Cowboys4Kids 

#PWI500 

#WAWFChat 

#NCBD 

#NationalBeachDay 

#Survivor 

#landscapechat 

#GrassrootsEd 

#1linewed 

#Rivals150 

